# ResNet18 Transfer Learning: Who's that Pokémon?

## Background and Understanding:

## Imports

In [16]:

import torch
import torchvision
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset
import torchvision.transforms as transforms

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
from dataset_class import PokemonImages

## Data Preperation

In [17]:
df = pd.read_csv('pokemon.csv')
df.head()

,national_number,gen,english_name,japanese_name,primary_type,secondary_type,classification,percent_male,percent_female,height_m,...,evochain_2,evochain_3,evochain_4,evochain_5,evochain_6,gigantamax,mega_evolution,mega_evolution_alt,description,image_path
0,1,I,bulbasaur,Fushigidane,grass,poison,Seed Pokémon,88.14,11.86,0.7,...,Ivysaur,Level,Venusaur,NaN,NaN,NaN,NaN,NaN,There is a plant seed on its back right from t...,pokemon_images/Bulbasaur.jpg
1,2,I,ivysaur,Fushigisou,grass,poison,Seed Pokémon,88.14,11.86,1.0,...,Ivysaur,Level,Venusaur,NaN,NaN,NaN,NaN,NaN,"When the bulb on its back grows large, it appe...",pokemon_images/Ivysaur.jpg
2,3,I,venusaur,Fushigibana,grass,poison,Seed Pokémon,88.14,11.86,2.0,...,Ivysaur,Level,Venusaur,NaN,NaN,Gigantamax Venusaur,Mega Venusaur,NaN,Its plant blooms when it is absorbing solar en...,pokemon_images/Venusaur.jpg
3,4,I,charmander,Hitokage,fire,NaN,Lizard Pokémon,88.14,11.86,0.6,...,Charmeleon,Level,Charizard,NaN,NaN,NaN,NaN,NaN,It has a preference for hot things. When it ra...,pokemon_images/Charmander.jpg
4,5,I,charmeleon,Lizardo,fire,NaN,Flame Pokémon,88.14,11.86,1.1,...,Charmeleon,Level,Charizard,NaN,NaN,NaN,NaN,NaN,"It has a barbaric nature. In battle, it whips ...",pokemon_images/Charmeleon.jpg


### Classifying a Pokémon's Primary type:

In [18]:
df = df.dropna(subset=['image_path', 'primary_type']) 
df['main_type'] = df['primary_type'].str.strip()  

In [19]:
type_names = df['main_type'].unique().tolist()
print("Classes:", type_names)

Classes: ['grass', 'fire', 'water', 'bug', 'normal', 'poison', 'electric', 'ground', 'fairy', 'fighting', 'psychic', 'rock', 'ghost', 'ice', 'dragon', 'dark', 'steel', 'flying']


In [24]:
label_encoder = LabelEncoder()
df['encoded_type'] = label_encoder.fit_transform(df['main_type'])
type_encoded = df['encoded_type'].unique().tolist()
print("Classes (Encoded):", type_encoded)

Classes (Encoded): [9, 6, 17, 0, 12, 13, 3, 10, 4, 5, 14, 15, 8, 11, 2, 1, 16, 7]


### Loading Data

In [21]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [22]:
dataset = PokemonImages(csv_file = "pokemon.csv", label = "main_type", transform = transforms)

## Pre-Trained ResNet18

In [23]:
resnet18 = models.resnet18(weights = True)
resnet18

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Freezing the CNN 